In [1]:
import pandas as pd
import pickle
import numpy as np
import re
from sklearn import linear_model

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn import ensemble 
from sklearn.metrics import mean_squared_error


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import xgboost as xgb
## even though xgb is not in the scikit learning package,
## it is a power model for both linear regression and tree model 

from sklearn.model_selection import train_test_split


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, Normalizer, PolynomialFeatures
import os
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification

from imblearn.over_sampling import SMOTE

from collections import Counter

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
os.chdir(r"C:\Users\13479\Desktop\dataset")

train = pd.read_csv('manual_train.csv') # parse_dates is used for non-standatd time data
test= pd.read_csv('manual_test.csv')

In [4]:
train=train.fillna(0)
test=test.fillna(0)

In [5]:
train.head()

,ym2888,ym2592,ym2468,ym2281,ym2158,ym2074,ym2070,ym2065,ym2061,ym2058,...,CD018,CD014,CD002,AP007,AP006,AP004,AP003,AP002,AP001,bad30
0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0396,0.0611,0.0006,...,1276.0,59789.0,2268.0,1,ios,12,4.0,1,29,0.0
1,0.0,18.0,0.0,0.0,0.0,37.0,1.0,0.0000,0.0000,0.0015,...,65.0,8622.0,254.0,1,android,12,3.0,2,24,1.0
2,1.0,0.0,0.0,2.0,0.0,16.0,0.0,0.0000,0.0067,0.0027,...,210.0,7559.0,276.0,1,ios,12,3.0,1,32,0.0
3,0.0,0.0,0.0,0.0,0.0,33.0,1.0,0.0035,0.0011,0.0012,...,96.0,14095.0,267.0,1,ios,12,1.0,1,43,0.0
4,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0000,0.0000,0.0000,...,205.0,37725.0,513.0,1,ios,6,1.0,1,24,0.0


In [5]:
train = train.drop(['AP006','MB006'], axis=1)
test = test.drop(['AP006','MB006'], axis=1)

In [6]:
def ks_statistic(Y,Y_hat):
	data = {"Y":Y,"Y_hat":Y_hat}
	df = pd.DataFrame(data)
	bins = np.array([-0.1,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
	category = pd.cut(df["Y_hat"],bins=bins)
	category = category.sort_values()
	#max_index = len(np.unique(df["Y_hat"]))
	Y = df.ix[category.index,:]['Y']
	Y_hat = df.ix[category.index,:]['Y_hat']
	df2 = pd.concat([Y,Y_hat],axis=1)
	df3 = pd.pivot_table(df2,values = ['Y_hat'],index ='Y_hat',columns='Y',aggfunc=len,fill_value=0)
	df4 = np.cumsum(df3)
	df5 = df4/df4.iloc[:,1].max()
	ks = max(abs(df5.iloc[:,0] - df5.iloc[:,1]))
	return ks/len(bins)

In [7]:
def ks_calc_cross(data,pred,y_label):
	crossfreq = pd.crosstab(data[pred[0]],data[y_label[0]])
	crossdens = crossfreq.cumsum(axis=0) / crossfreq.sum()
	crossdens['gap'] = abs(crossdens[0] - crossdens[1])
	ks = crossdens[crossdens['gap'] == crossdens['gap'].max()]
	return ks,crossdens

In [8]:
def print_acc(test_x, test_y, model):
	print ('Accuracy of Classifier:%f'%model.score(test_x, test_y))

In [10]:
def array_trans(x, y): 
    # x: prediction
    # y: actual value
    a = np.array(x) 
    b = np.array(y)
    result = np.vstack((a,b))
    
    return result  

def cal_rate(result, thres):
    all_number = len(result[0])
    # print all_number
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    for item in range(all_number):
        prediction = result[0][item]
        if prediction >= thres:
            prediction = 1
        if prediction == 1:
            if result[1][item] == 1:
                TP += 1
            else:
                FP += 1
        else:
            if result[1][item] == 0:
                TN += 1
            else:
                FN += 1
    # print TP+FP+TN+FN
    accracy = float(TP+TN) / float(all_number)
    if TP+FP == 0:
        precision = 0
    else:
        precision = float(TP) / float(TP+FP)
    TPR = float(TP) / float(TP+FN)
    TNR = float(TN) / float(FP+TN)
    FNR = float(FN) / float(TP+FN)
    FPR = float(FP) / float(FP+TN)
    print('accuracy')
    print(accracy)
    print('precision')
    print(precision)
    print('sensitivity')
    print(TPR)
    print('specificity')
    print(TNR)
    # print accracy, precision, TPR, TNR, FNR, FPR
    return accracy, precision, TPR, TNR, FNR, FPR

def AUROC_func(result):
    prob = np.hstack((np.random.rand(98),[0,1]))
    threshold_vaule = sorted(prob)
    threshold_num = len(threshold_vaule)
    #get a array for plot  
    accracy_array = np.zeros(threshold_num)
    precision_array = np.zeros(threshold_num)
    TPR_array = np.zeros(threshold_num)
    TNR_array = np.zeros(threshold_num)
    FNR_array = np.zeros(threshold_num)
    FPR_array = np.zeros(threshold_num)

    #calculate all the rates
    for thres in range(threshold_num):
        accracy_array[thres] = np.array(cal_rate(result, threshold_vaule[thres]))[0]
        precision_array[thres] = np.array(cal_rate(result, threshold_vaule[thres]))[1]
        TPR_array[thres] = np.array(cal_rate(result, threshold_vaule[thres]))[2]
        TNR_array[thres] = np.array(cal_rate(result, threshold_vaule[thres]))[3]
        FNR_array[thres] = np.array(cal_rate(result, threshold_vaule[thres]))[4]
        FPR_array[thres] = np.array(cal_rate(result, threshold_vaule[thres]))[5]
        print('threshold #: %d, threshold: %f' % (thres+1,threshold_vaule[thres]))

    #draw the plot with the rates array

    AUC = np.trapz(-TPR_array, FPR_array)
    threshold = np.argmin(abs(FNR_array - FPR_array))
    EER = (FNR_array[threshold]+FPR_array[threshold])/2
    plt.plot(FPR_array, TPR_array)
    plt.title('roc')
    plt.xlabel('FPR_array')
    plt.ylabel('TPR_array')
    plt.show()
    
    return AUC

In [53]:
def createGains(Y,Yhat):
    data = {"Y":Y_test['bad30'],'Y_hat':Yhat}
    test_scores = pd.DataFrame(data)
    test_scores.columns = ['true', 'predict']

    #sort on prediction (descending), add id, and decile for groups containing 1/10 of datapoints
    test_scores = test_scores.sort_values(by='predict',ascending=False)
    test_scores['row_id'] = range(0,0+len(test_scores))
    test_scores['decile'] = ( test_scores['row_id'] / (len(test_scores)/10) ).astype(int)
    #see count by decile
    test_scores.loc[test_scores['decile'] == 10]=9
    test_scores['decile'].value_counts()

    #create gains table
    gains = test_scores.groupby('decile')['true'].agg(['count','sum'])
    gains.columns = ['count','actual']
    gains

    #add features to gains table
    gains['non_actual'] = gains['count'] - gains['actual']
    gains['cum_count'] = gains['count'].cumsum()
    gains['cum_actual'] = gains['actual'].cumsum()
    gains['cum_non_actual'] = gains['non_actual'].cumsum()
    gains['percent_cum_actual'] = (gains['cum_actual'] / np.max(gains['cum_actual'])).round(2)
    gains['percent_cum_non_actual'] = (gains['cum_non_actual'] / np.max(gains['cum_non_actual'])).round(2)
    gains['if_random'] = np.max(gains['cum_actual']) /10 
    gains['if_random'] = gains['if_random'].cumsum()
    gains['lift'] = (gains['cum_actual'] / gains['if_random']).round(2)
    gains['K_S'] = np.abs( gains['percent_cum_actual'] -  gains['percent_cum_non_actual'] ) * 100
    gains['gain']=(gains['cum_actual']/gains['cum_count']*100).round(2)
    gains = pd.DataFrame(gains)
    return(gains)

In [13]:
X_train = train.drop(['bad30'], axis=1)
Y_train = train['bad30']
X_test = test.drop(['bad30'], axis=1)
Y_test = test['bad30']
Y_test = Y_test.reset_index().drop('index',axis = 1)

In [57]:
def lasso_autotune(X,y):
	
	param_test = {
	 'alpha':[1e-10, 1e-8, 1e-4,1e-2, 1, 5, 10, 100, 500, 1000] #set different levels of penalties and substitute into lasso model 
	}
   ##we run the cross validation 5 time for each penalty level and find the best tune
   ## refit = True allows to put best parameters into lasso and get the best-fit model  
	gsearch = GridSearchCV(Lasso(fit_intercept=True, normalize=True, selection='random'),refit=True,
	                        param_grid = param_test, scoring='neg_mean_absolute_error',iid=True, cv=5)
    
	gsearch.fit(X,y)
	print('____________________________________________')
	print('tune lasso')
	print('____________________________________________')
	print(gsearch.cv_results_)
	print(gsearch.best_params_)
	print(gsearch.best_score_)
	print('ks_statistic: ', ks_statistic(y,gsearch.predict(X)))
	return gsearch.best_estimator_

In [58]:
lasso_param = lasso_autotune(X_train, Y_train)
print('='*15)
print('lasso test score')
print('ks_statistic: ', ks_statistic(Y_test['bad30'],lasso_param.predict(X_test)))
print('='*15)

____________________________________________
tune lasso
____________________________________________
{'mean_fit_time': array([2.5488874 , 1.64322152, 0.31785688, 0.31096611, 0.30935407,
       0.31266956, 0.31371574, 0.29442925, 0.3232688 , 0.32168474]), 'std_fit_time': array([0.28422694, 0.21579969, 0.00284565, 0.00776935, 0.01412105,
       0.00670276, 0.00576156, 0.01187341, 0.01368807, 0.01053179]), 'mean_score_time': array([0.02134342, 0.02254009, 0.01985645, 0.01894956, 0.02154551,
       0.02284136, 0.02313762, 0.01875   , 0.02134385, 0.02154651]), 'std_score_time': array([0.00518609, 0.00360145, 0.00274535, 0.00327757, 0.00306676,
       0.00354462, 0.00341973, 0.00193335, 0.00232565, 0.00336956]), 'param_alpha': masked_array(data=[1e-10, 1e-08, 0.0001, 0.01, 1, 5, 10, 100, 500, 1000],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 1e-10}, {'alpha

C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
C:\Users\13479\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':

ks_statistic:  0.29529936974789917
lasso test score


C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
C:\Users\13479\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':

ks_statistic:  0.29436435124508525


In [59]:
roc_auc_score(Y_test['bad30'],lasso_param.predict(X_test))

0.6629888788948024

In [60]:
createGains(Y_test['bad30'],lasso_param.predict(X_test))

,count,actual,non_actual,cum_count,cum_actual,cum_non_actual,percent_cum_actual,percent_cum_non_actual,if_random,lift,K_S,gain
decile,,,,,,,,,,,,
0,1999,661.0,1338.0,1999,661.0,1338.0,0.17,0.08,381.5,1.73,9.0,33.07
1,1998,580.0,1418.0,3997,1241.0,2756.0,0.33,0.17,763.0,1.63,16.0,31.05
2,1998,538.0,1460.0,5995,1779.0,4216.0,0.47,0.26,1144.5,1.55,21.0,29.67
3,1999,440.0,1559.0,7994,2219.0,5775.0,0.58,0.36,1526.0,1.45,22.0,27.76
4,1998,404.0,1594.0,9992,2623.0,7369.0,0.69,0.46,1907.5,1.38,23.0,26.25
5,1998,370.0,1628.0,11990,2993.0,8997.0,0.78,0.56,2289.0,1.31,22.0,24.96
6,1999,313.0,1686.0,13989,3306.0,10683.0,0.87,0.66,2670.5,1.24,21.0,23.63
7,1998,263.0,1735.0,15987,3569.0,12418.0,0.94,0.77,3052.0,1.17,17.0,22.32
8,1998,160.0,1838.0,17985,3729.0,14256.0,0.98,0.88,3433.5,1.09,10.0,20.73


In [24]:
def ridge_autotune(X,y):
	
	param_test = {
	 'alpha':[1e-10, 1e-8, 1e-4,1e-2, 1, 5, 10, 100],
	 'solver':['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
	}

	gsearch = GridSearchCV(Ridge(),refit=True,
	                        param_grid = param_test, scoring='neg_mean_absolute_error',iid=True, cv=5)
	gsearch.fit(X,y)
	print('____________________________________________')
	print('tune ridge')
	print('____________________________________________')
	print(gsearch.cv_results_)
	print(gsearch.best_params_)
	print(gsearch.best_score_)
	print(ks_statistic(gsearch.predict(X),y))

	return gsearch.best_estimator_

In [26]:
ridge_param = ridge_autotune(X_train,Y_train)
print('='*15)
print('ridge test score')
print('ks_statistic: ', ks_statistic(Y_test['bad30'],ridge_param.predict(X_test)))
print('='*15)

____________________________________________
tune ridge
____________________________________________
{'mean_fit_time': array([ 0.20485177,  0.45538173,  0.21123524,  0.18710165,  0.34985838,
       60.6587513 , 71.4405458 ,  0.19487872,  0.46854758,  0.20385475,
        0.17174144,  0.37719507, 60.96168666, 86.98515553,  0.22490516,
        0.54329481,  0.23846197,  0.23348293,  0.46022406, 69.89666176,
       80.51625772,  0.23199248,  0.59234247,  0.24066119,  0.24648094,
        0.46775279, 70.40808916, 80.55457335,  0.2431571 ,  0.62883859,
        0.2550818 ,  0.24115858,  0.46365538, 67.71954484, 81.27643542,
        0.22849598,  0.54511533,  0.22232919,  0.23058877,  0.45051584,
       68.38036933, 76.30807066,  0.23278093,  0.5295167 ,  0.22960572,
        0.21783872,  0.4537921 , 67.51474714, 74.10936003,  0.21476088,
        0.50073905,  0.2085494 ,  0.21203413,  0.45780921, 61.56576552,
       71.56058164]), 'std_fit_time': array([1.98021799e-02, 1.34037325e-02, 4.69531528e-

C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
C:\Users\13479\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':

0.0
ridge test score
ks_statistic:  0.29436435124508525


C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
C:\Users\13479\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':

In [27]:
roc_auc_score(Y_test['bad30'],ridge_param.predict(X_test))

0.6629814697964946

In [54]:
createGains(Y_test['bad30'],ridge_param.predict(X_test))

,count,actual,non_actual,cum_count,cum_actual,cum_non_actual,percent_cum_actual,percent_cum_non_actual,if_random,lift,K_S,gain
decile,,,,,,,,,,,,
0,1999,661.0,1338.0,1999,661.0,1338.0,0.17,0.08,381.5,1.73,9.0,33.07
1,1998,580.0,1418.0,3997,1241.0,2756.0,0.33,0.17,763.0,1.63,16.0,31.05
2,1998,537.0,1461.0,5995,1778.0,4217.0,0.47,0.26,1144.5,1.55,21.0,29.66
3,1999,442.0,1557.0,7994,2220.0,5774.0,0.58,0.36,1526.0,1.45,22.0,27.77
4,1998,403.0,1595.0,9992,2623.0,7369.0,0.69,0.46,1907.5,1.38,23.0,26.25
5,1998,370.0,1628.0,11990,2993.0,8997.0,0.78,0.56,2289.0,1.31,22.0,24.96
6,1999,313.0,1686.0,13989,3306.0,10683.0,0.87,0.66,2670.5,1.24,21.0,23.63
7,1998,263.0,1735.0,15987,3569.0,12418.0,0.94,0.77,3052.0,1.17,17.0,22.32
8,1998,160.0,1838.0,17985,3729.0,14256.0,0.98,0.88,3433.5,1.09,10.0,20.73


In [15]:
def random_forest_autotune(X,y):

	param_test = {
	 'max_depth':[5,10,13],## set different heights for each tree 
	 'max_features':['auto','sqrt'], ##the maximum amount of features for each tree is limited to square root of the total amount of features in the original dataset
	 'n_estimators': [200,400,600] ## set different numbers of tree to build in each model 
	}
    
  ## the process of cross validation for random forest is the same as lasso, but more complex 
  ##since we have 3 levels of depth and number of trees so that the total number of running time is 3*3*5=45 runs
    
	gsearch = GridSearchCV(RandomForestRegressor(),refit=True,
	                        param_grid = param_test, scoring='neg_mean_absolute_error',iid=True, cv=5)
	gsearch.fit(X,y)
	print('____________________________________________')
	print('tune random forest')
	print('____________________________________________')
	print(gsearch.cv_results_)
	print(gsearch.best_params_)
	print(gsearch.best_score_)
	print(ks_statistic(gsearch.predict(X),y))

	return gsearch.best_estimator_

In [17]:
random_forest_param = random_forest_autotune(X_train, Y_train)
print('='*15)
print('random forest test score')
print('ks_statistic: ', ks_statistic(Y_test['bad30'],random_forest_param.predict(X_test)))
print('='*15)

____________________________________________
tune random forest
____________________________________________
{'mean_fit_time': array([ 164.62300415,  328.04364367,  491.95872002,   13.95226345,
         27.73436809,   41.57797275,  344.08700366,  688.97703338,
       1032.48317871,   28.9230001 ,   57.73539128,   86.66933212,
        450.37123857,  900.04461899, 1394.72956309,   42.04180942,
         83.11603651,  124.80244961]), 'std_fit_time': array([5.61939745e-01, 3.54067701e-01, 4.38475536e-01, 5.64393116e-02,
       9.33955804e-02, 1.50310991e-01, 2.79140533e-01, 6.69236363e-01,
       9.43145959e-01, 7.59248828e-02, 6.40442369e-02, 2.28543149e-01,
       6.10505015e-01, 8.32166432e-01, 6.78813864e+01, 5.05567450e-01,
       9.12567510e-01, 8.60805317e-01]), 'mean_score_time': array([0.11309996, 0.2078382 , 0.30239086, 0.11230636, 0.20664158,
       0.30139551, 0.25990415, 0.49268966, 0.73502798, 0.27826061,
       0.52539515, 0.79418516, 0.33765302, 0.65625186, 1.10027924,
     

C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
C:\Users\13479\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':

0.0
random forest test score
ks_statistic:  0.2945311569164781


C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
C:\Users\13479\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':

In [28]:
roc_auc_score(Y_test['bad30'],random_forest_param.predict(X_test))

0.6629295898958705

In [55]:
createGains(Y_test['bad30'],random_forest_param.predict(X_test))

,count,actual,non_actual,cum_count,cum_actual,cum_non_actual,percent_cum_actual,percent_cum_non_actual,if_random,lift,K_S,gain
decile,,,,,,,,,,,,
0,1999,671.0,1328.0,1999,671.0,1328.0,0.18,0.08,381.5,1.76,10.0,33.57
1,1998,592.0,1406.0,3997,1263.0,2734.0,0.33,0.17,763.0,1.66,16.0,31.60
2,1998,524.0,1474.0,5995,1787.0,4208.0,0.47,0.26,1144.5,1.56,21.0,29.81
3,1999,468.0,1531.0,7994,2255.0,5739.0,0.59,0.35,1526.0,1.48,24.0,28.21
4,1998,381.0,1617.0,9992,2636.0,7356.0,0.69,0.45,1907.5,1.38,24.0,26.38
5,1998,362.0,1636.0,11990,2998.0,8992.0,0.79,0.56,2289.0,1.31,23.0,25.00
6,1999,290.0,1709.0,13989,3288.0,10701.0,0.86,0.66,2670.5,1.23,20.0,23.50
7,1998,252.0,1746.0,15987,3540.0,12447.0,0.93,0.77,3052.0,1.16,16.0,22.14
8,1998,179.0,1819.0,17985,3719.0,14266.0,0.97,0.88,3433.5,1.08,9.0,20.68


In [71]:
def xgb_autotune(X,y):
	param = {
	        'objective':'reg:linear', # use linear regression
	        'n_estimators':500, # number of regression
	        'learning_rate':0.05, #Step shrinkage used in update to prevents overfitting. 
	        'gamma':0, # Minimum loss reduction required to make a further partition on a leaf node 
                       #of the tree. The larger gamma is, the more conservative the algorithm will be
	        'max_depth':5,# maximum height for each regression, increasing depth means more likely of overfitting
	        'min_child_weight':1, #Minimum sum of instance weight (hessian) needed in a child. 
                                 #If the tree partition step results in a leaf node with the sum of instance 
                                 #weight less than min_child_weight, then the building process will stop further partitioning
	        'colsample_bytree':0.5,
	        'subsample': 0.8, 
	        'verbose':1,
	        'reg_alpha':0} # alpha approaches to 1 means more consertive for creating each new regression


	param_test1 = {
	 'max_depth':[5,7,13],
	 'reg_alpha':[0.1,1,10],
	 'learning_rate':[0.05,0.1],    #set different level of parameter for cross valiadation 
	}
 ## there are 3*3*2*5=90 runs for cross validation
 ## the geenral concept is the same as lasso
	gsearch = GridSearchCV(xgb.XGBRegressor(objective=param['objective'],
	                          n_estimators=param['n_estimators'], learning_rate = param['learning_rate'], gamma=param['gamma'],
	                          colsample_bytree=param['colsample_bytree'], subsample=param['subsample'], reg_alpha=param['reg_alpha']
	                          ),
	                        param_grid = param_test1, scoring='neg_mean_absolute_error',iid=False, cv=5)
	gsearch.fit(X,y)
	print('____________________________________________')
	print('tune max_depth, min_child_weight')
	print('____________________________________________')
	print(gsearch.cv_results_)
	print(gsearch.best_params_)
	print(gsearch.best_score_)
	print(ks_statistic(gsearch.predict(X),y))

	return gsearch.best_estimator_
    

In [19]:
xgb_param = xgb_autotune(X_train,Y_train)
print('='*15)
print('xgb test score')
print('ks_statistic: ', ks_statistic(Y_test['bad30'],xgb_param.predict(X_test)))
print('='*15)

C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:28:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:31:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

[21:07:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[21:11:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
____________________________________________
tune max_depth, min_child_weight
____________________________________________
{'mean_fit_time': array([179.77452545, 182.54762726, 187.98064685, 156.20256267,
       259.71082749, 267.18730502, 293.79204245, 214.63280983,
       683.27478337, 640.2024869 , 590.33417211, 304.73367801,
       179.39398613, 182.62718983, 187.81179028, 131.4291676 ,
       261.33722095, 273.10487957, 272.35594802, 150.14718294,
       575.46042995, 564.23902869, 516.59444757, 214.91344471]), 'std_fit_time': array([ 0.45850391,  0.17432048,  3.51509592,  1.69126473,  1.78465568,
        2.16637682, 14.02199694,  3.96750139, 35.92514141,  7.84795893,
       22.48895067,  1.5222999 ,  1.95608127,  0.94726538,  0.94972897,
        1.26927391,  2.09859346,  0.97068962,  5.59852643,  1.23973725,
        0.85465355,  0.33011971,  5.66733516,  1.89668907])

C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
C:\Users\13479\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':

0.0
xgb test score
ks_statistic:  0.2962707017752889


In [29]:
roc_auc_score(Y_test['bad30'],xgb_param.predict(X_test))

0.6562756116478159

In [56]:
createGains(Y_test['bad30'],xgb_param.predict(X_test))

,count,actual,non_actual,cum_count,cum_actual,cum_non_actual,percent_cum_actual,percent_cum_non_actual,if_random,lift,K_S,gain
decile,,,,,,,,,,,,
0,1999,728.0,1271.0,1999,728.0,1271.0,0.19,0.08,381.5,1.91,11.0,36.42
1,1998,558.0,1440.0,3997,1286.0,2711.0,0.34,0.17,763.0,1.69,17.0,32.17
2,1998,464.0,1534.0,5995,1750.0,4245.0,0.46,0.26,1144.5,1.53,20.0,29.19
3,1999,463.0,1536.0,7994,2213.0,5781.0,0.58,0.36,1526.0,1.45,22.0,27.68
4,1998,387.0,1611.0,9992,2600.0,7392.0,0.68,0.46,1907.5,1.36,22.0,26.02
5,1998,338.0,1660.0,11990,2938.0,9052.0,0.77,0.56,2289.0,1.28,21.0,24.50
6,1999,299.0,1700.0,13989,3237.0,10752.0,0.85,0.67,2670.5,1.21,18.0,23.14
7,1998,256.0,1742.0,15987,3493.0,12494.0,0.92,0.77,3052.0,1.14,15.0,21.85
8,1998,185.0,1813.0,17985,3678.0,14307.0,0.96,0.88,3433.5,1.07,8.0,20.45


In [72]:
xgb_param = xgb_autotune(X_train,Y_train)
print('='*15)
print('xgb test score')
print('ks_statistic: ', ks_statistic(Y_test['bad30'],xgb_param.predict(X_test)))
print('='*15)

C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:18:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:21:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

[05:43:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[05:48:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
____________________________________________
tune max_depth, min_child_weight
____________________________________________
{'mean_fit_time': array([ 98.08737841,  93.59494977,  95.30126171, 131.98271499,
       135.44488854, 138.44222775, 325.86797256, 334.91880851,
       283.31989703,  82.73082542,  83.89378824,  86.06643953,
       119.1862041 , 123.02317104, 127.49302239, 301.6605001 ,
       301.45452714, 272.5656889 ]), 'std_fit_time': array([16.98923577,  0.37798782,  0.79045124,  0.39722497,  1.09904225,
        1.1377527 ,  1.91426568,  3.24788594,  4.78217926,  0.20902058,
        0.14906013,  0.13626478,  0.10204175,  0.29762433,  0.20897345,
        0.92041781,  0.30448988,  0.26813678]), 'mean_score_time': array([0.40363784, 0.43238564, 0.47646027, 0.60930557, 0.66368008,
       0.70421548, 3.32051067, 3.27150888, 1.57778072, 0.38955297,
       0.40231123, 0.

C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
C:\Users\13479\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':

0.0
xgb test score
ks_statistic:  0.2955796497080901


In [73]:
roc_auc_score(Y_test['bad30'],xgb_param.predict(X_test))

0.6624005040780846

In [74]:
createGains(Y_test['bad30'],xgb_param.predict(X_test))

,count,actual,non_actual,cum_count,cum_actual,cum_non_actual,percent_cum_actual,percent_cum_non_actual,if_random,lift,K_S,gain
decile,,,,,,,,,,,,
0,1999,743.0,1256.0,1999,743.0,1256.0,0.19,0.08,381.5,1.95,11.0,37.17
1,1998,547.0,1451.0,3997,1290.0,2707.0,0.34,0.17,763.0,1.69,17.0,32.27
2,1998,482.0,1516.0,5995,1772.0,4223.0,0.46,0.26,1144.5,1.55,20.0,29.56
3,1999,455.0,1544.0,7994,2227.0,5767.0,0.58,0.36,1526.0,1.46,22.0,27.86
4,1998,404.0,1594.0,9992,2631.0,7361.0,0.69,0.46,1907.5,1.38,23.0,26.33
5,1998,356.0,1642.0,11990,2987.0,9003.0,0.78,0.56,2289.0,1.30,22.0,24.91
6,1999,286.0,1713.0,13989,3273.0,10716.0,0.86,0.66,2670.5,1.23,20.0,23.40
7,1998,244.0,1754.0,15987,3517.0,12470.0,0.92,0.77,3052.0,1.15,15.0,22.00
8,1998,166.0,1832.0,17985,3683.0,14302.0,0.97,0.88,3433.5,1.07,9.0,20.48


In [94]:
L = []
for i in xgb_param.predict(X_test):
    if i > 0.14:
        L.append(1)
    else:
        L.append(0)
ypred = L

In [95]:
roc_auc_score(Y_test['bad30'],ypred)

0.6069295253702441

In [96]:
createGains(Y_test['bad30'],ypred)

,count,actual,non_actual,cum_count,cum_actual,cum_non_actual,percent_cum_actual,percent_cum_non_actual,if_random,lift,K_S,gain
decile,,,,,,,,,,,,
0,1999,484.0,1515.0,1999,484.0,1515.0,0.13,0.09,381.5,1.27,4.0,24.21
1,1998,473.0,1525.0,3997,957.0,3040.0,0.25,0.19,763.0,1.25,6.0,23.94
2,1998,469.0,1529.0,5995,1426.0,4569.0,0.37,0.28,1144.5,1.25,9.0,23.79
3,1999,476.0,1523.0,7994,1902.0,6092.0,0.50,0.38,1526.0,1.25,12.0,23.79
4,1998,519.0,1479.0,9992,2421.0,7571.0,0.63,0.47,1907.5,1.27,16.0,24.23
5,1998,478.0,1520.0,11990,2899.0,9091.0,0.76,0.56,2289.0,1.27,20.0,24.18
6,1999,355.0,1644.0,13989,3254.0,10735.0,0.85,0.66,2670.5,1.22,19.0,23.26
7,1998,186.0,1812.0,15987,3440.0,12547.0,0.90,0.78,3052.0,1.13,12.0,21.52
8,1998,190.0,1808.0,17985,3630.0,14355.0,0.95,0.89,3433.5,1.06,6.0,20.18


In [75]:
predict_met = pd.DataFrame(data={method: [np.nan] * X_train.shape[0] for method in ['lasso', 'ridge', 
                                                                                  'xgb', 'random forest']})

In [76]:
predict_met['lasso'] = lasso_param.predict(X_train)
predict_met['ridge'] = ridge_param.predict(X_train)
predict_met['random forest'] = random_forest_param.predict(X_train)
predict_met['xgb'] = xgb_param.predict(X_train)

In [77]:
ensenble_xgb = xgb_autotune(predict_met,Y_train)

C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:38:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:39:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

[11:00:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\13479\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[11:00:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
____________________________________________
tune max_depth, min_child_weight
____________________________________________
{'mean_fit_time': array([ 8.78271213,  8.92813854,  8.11934381, 12.07990499, 11.80404315,
        9.49947534, 21.56251903, 15.78625221, 10.0793859 ,  9.12050242,
        9.1943346 ,  7.98626404, 12.56486793, 12.05208702,  9.10063844,
       22.56515794, 15.80039792,  9.16428866]), 'std_fit_time': array([0.32051598, 0.16985388, 0.2227118 , 0.07904095, 0.21343449,
       0.33338391, 0.45613301, 0.74491963, 0.30003123, 0.0992992 ,
       0.13214172, 0.44466753, 0.15988782, 0.36666941, 0.68255456,
       0.67331719, 1.08474827, 0.7271098 ]), 'mean_score_time': array([0.28542428, 0.27501483, 0.23828425, 0.47542238, 0.41994481,
       0.27875896, 1.59723773, 0.55902939, 0.29403653, 0.27586656,
       0.28036542, 0.22932091, 0.48125229, 0.42867765, 0.2710653

C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
C:\Users\13479\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
C:\Users\13479\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':

0.0


In [78]:
test_met = pd.DataFrame(data={method: [np.nan] * X_test.shape[0] for method in ['lasso',  'ridge',
                                                                                  'xgb', 'random forest'
                                                                                  ]})
test_met['lasso'] = lasso_param.predict(X_test)
test_met['ridge'] = ridge_param.predict(X_test)
test_met['xgb'] = xgb_param.predict(X_test)
test_met['random forest'] = random_forest_param.predict(X_test)

In [79]:
roc_auc_score(Y_test['bad30'],ensenble_xgb.predict(test_met))

0.5748000110893288

In [70]:
createGains(Y_test['bad30'],ensenble_xgb.predict(test_met))

,count,actual,non_actual,cum_count,cum_actual,cum_non_actual,percent_cum_actual,percent_cum_non_actual,if_random,lift,K_S,gain
decile,,,,,,,,,,,,
0,1999,710.0,1289.0,1999,710.0,1289.0,0.19,0.08,381.5,1.86,11.0,35.52
1,1998,502.0,1496.0,3997,1212.0,2785.0,0.32,0.17,763.0,1.59,15.0,30.32
2,1998,386.0,1612.0,5995,1598.0,4397.0,0.42,0.27,1144.5,1.40,15.0,26.66
3,1999,340.0,1659.0,7994,1938.0,6056.0,0.51,0.37,1526.0,1.27,14.0,24.24
4,1998,242.0,1756.0,9992,2180.0,7812.0,0.57,0.48,1907.5,1.14,9.0,21.82
5,1998,202.0,1796.0,11990,2382.0,9608.0,0.62,0.59,2289.0,1.04,3.0,19.87
6,1999,254.0,1745.0,13989,2636.0,11353.0,0.69,0.70,2670.5,0.99,1.0,18.84
7,1998,289.0,1709.0,15987,2925.0,13062.0,0.77,0.81,3052.0,0.96,4.0,18.30
8,1998,389.0,1609.0,17985,3314.0,14671.0,0.87,0.91,3433.5,0.97,4.0,18.43
